In [384]:
import pandas as pd

In [385]:
df = pd.read_csv('fruit.csv')

In [386]:
df

,color,diameter,target
0,green,3,Apple
1,yellow,3,Apple
2,red,1,Grape
3,red,1,Grape
4,yellow,3,Lemon


In [387]:
headers = df.columns

In [388]:
headers

Index(['color', 'diameter', 'target'], dtype='object')

In [389]:
headers[0]

'color'

In [390]:
training_data = df
training_data = [l.tolist() for l in training_data.values]
training_data

[['green', 3, 'Apple'],
 ['yellow', 3, 'Apple'],
 ['red', 1, 'Grape'],
 ['red', 1, 'Grape'],
 ['yellow', 3, 'Lemon']]

In [391]:
def unique_vals(rows, col):
    values = []
    for r in rows:
        values.append(r[col])
    return set(values)
#     return set([row[col] for row in rows])

In [392]:
training_data

[['green', 3, 'Apple'],
 ['yellow', 3, 'Apple'],
 ['red', 1, 'Grape'],
 ['red', 1, 'Grape'],
 ['yellow', 3, 'Lemon']]

In [393]:
unique_vals(training_data, 2)
# print(training_data[:][])

{'Apple', 'Grape', 'Lemon'}

In [394]:
unique_vals(training_data, 0)

{'green', 'red', 'yellow'}

In [395]:
unique_vals(training_data, 1)

{1, 3}

In [396]:
def class_counts(rows, col):
    labels = unique_vals(rows, col)
    counts = {}
    for label in labels:
        counts[label] = 0
    for row in rows:
        counts[row[col]] += 1
    return counts

In [397]:
class_counts(training_data, 2)

{'Lemon': 1, 'Apple': 2, 'Grape': 2}

In [398]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [399]:
is_numeric(7)


True

In [400]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            headers[self.column], condition, str(self.value))

In [401]:
Question(1, 3)

Is diameter >= 3?

In [402]:
q = Question(0, 'green')
q

Is color == green?

In [403]:
example = training_data[0]

q.match(example)

True

In [404]:
example = training_data[1]

q.match(example)

False

In [405]:
example = training_data[2]
q.match(example)

False

In [406]:
example = training_data[3]

q.match(example)

False

In [407]:
example = training_data[4]
q.match(example)

False

In [408]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
#             [l.tolist() for l in list1]
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [409]:
true_rows, false_rows = partition(training_data, Question(0, 'red'))


In [410]:
true_rows

[['red', 1, 'Grape'], ['red', 1, 'Grape']]

In [411]:
false_rows

[['green', 3, 'Apple'], ['yellow', 3, 'Apple'], ['yellow', 3, 'Lemon']]

In [412]:
def gini(rows):
    counts = class_counts(rows, 2)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [413]:
example = training_data
print(example)
gini(example)

[['green', 3, 'Apple'], ['yellow', 3, 'Apple'], ['red', 1, 'Grape'], ['red', 1, 'Grape'], ['yellow', 3, 'Lemon']]


0.6399999999999999

In [414]:
example = training_data[:2]
print(example)
gini(example)


[['green', 3, 'Apple'], ['yellow', 3, 'Apple']]


0.0

In [415]:
example = training_data[1:3]
print(example)
gini(example)

[['yellow', 3, 'Apple'], ['red', 1, 'Grape']]


0.5

In [416]:
example = training_data[1:4]
print(example)
gini(example)

[['yellow', 3, 'Apple'], ['red', 1, 'Grape'], ['red', 1, 'Grape']]


0.4444444444444444

In [417]:
example = training_data[2:6]
print(example)
gini(example)

[['red', 1, 'Grape'], ['red', 1, 'Grape'], ['yellow', 3, 'Lemon']]


0.4444444444444444

In [418]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [419]:
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [420]:
true_rows, false_rows = partition(training_data, Question(0, 'green'))


In [421]:
print(true_rows)


[['green', 3, 'Apple']]


In [422]:
print(false_rows)

[['yellow', 3, 'Apple'], ['red', 1, 'Grape'], ['red', 1, 'Grape'], ['yellow', 3, 'Lemon']]


In [423]:
example

[['red', 1, 'Grape'], ['red', 1, 'Grape'], ['yellow', 3, 'Lemon']]

In [424]:
counts = class_counts(example, 2)
counts

{'Lemon': 1, 'Grape': 2}

In [425]:
true_rows

[['green', 3, 'Apple']]

In [426]:
false_rows

[['yellow', 3, 'Apple'],
 ['red', 1, 'Grape'],
 ['red', 1, 'Grape'],
 ['yellow', 3, 'Lemon']]

In [427]:
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [428]:
true_rows, false_rows = partition(training_data, Question(0, 'red'))

In [429]:
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

In [430]:
def find_best_split(rows):

    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [431]:
best_gain, best_question = find_best_split(training_data)
best_question

Is diameter >= 3?

In [432]:
class Leaf:

    def __init__(self, rows):
        self.predictions = class_counts(rows,2)

In [433]:
class Decision_Node:

    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [434]:
def build_tree(rows):
    gain, question = find_best_split(rows)

    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

In [435]:
def print_tree(node, spacing=""):

    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    print (spacing + str(node.question))

    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [436]:
my_tree = build_tree(training_data)


In [437]:
print_tree(my_tree)


Is diameter >= 3?
--> True:
  Is color == yellow?
  --> True:
    Predict {'Lemon': 1, 'Apple': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [438]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [439]:
classify(training_data[0], my_tree)


{'Apple': 1}

In [440]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [441]:
print_leaf(classify(training_data[0], my_tree))


{'Apple': '100%'}

In [442]:
print_leaf(classify(training_data[1], my_tree))


{'Lemon': '50%', 'Apple': '50%'}

In [444]:
for row in training_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Lemon': '50%', 'Apple': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Lemon': '50%', 'Apple': '50%'}
